In [1]:
import tensorflow as tf
from object_detection.utils import config_util
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import cv2 as cv
import numpy as np
from PIL import Image

In [2]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

In [3]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [4]:
image = Image.open("test_images/1-1.jpg")
image_np = np.asarray(image)
np.expand_dims(image_np, 0)

array([[[[ 90, 129, 124],
         [ 90, 129, 124],
         [ 90, 129, 124],
         ...,
         [ 49, 104,  99],
         [ 49, 104,  99],
         [ 48, 103,  98]],

        [[108, 144, 140],
         [109, 145, 141],
         [109, 145, 141],
         ...,
         [ 70, 119, 116],
         [ 69, 118, 115],
         [ 69, 118, 115]],

        [[124, 156, 153],
         [124, 156, 153],
         [124, 156, 153],
         ...,
         [ 89, 127, 128],
         [ 89, 127, 128],
         [ 88, 126, 127]],

        ...,

        [[ 97, 107, 109],
         [ 97, 107, 109],
         [ 97, 107, 109],
         ...,
         [ 44,  53,  48],
         [ 45,  54,  49],
         [ 45,  54,  49]],

        [[ 96, 106, 108],
         [ 96, 106, 108],
         [ 96, 106, 108],
         ...,
         [ 42,  51,  46],
         [ 42,  51,  46],
         [ 42,  51,  46]],

        [[ 96, 106, 108],
         [ 96, 106, 108],
         [ 96, 106, 108],
         ...,
         [ 40,  49,  44],
        

In [5]:
input_tensor = tf.convert_to_tensor(image_np, dtype=tf.float32)

detections = detect_fn(input_tensor)

ValueError: in user code:

    File "C:\Users\beesh\AppData\Local\Temp/ipykernel_14256/2522897421.py", line 11, in detect_fn  *
        image, shapes = detection_model.preprocess(image)
    File "c:\Users\beesh\anaconda3\lib\site-packages\object_detection\meta_architectures\ssd_meta_arch.py", line 484, in preprocess  *
        normalized_inputs, self._image_resizer_fn)
    File "c:\Users\beesh\anaconda3\lib\site-packages\object_detection\utils\shape_utils.py", line 492, in resize_images_and_return_shapes  *
        outputs = static_or_dynamic_map_fn(
    File "c:\Users\beesh\anaconda3\lib\site-packages\object_detection\utils\shape_utils.py", line 246, in static_or_dynamic_map_fn  *
        outputs = [fn(arg) for arg in tf.unstack(elems)]
    File "c:\Users\beesh\anaconda3\lib\site-packages\object_detection\core\preprocessor.py", line 3330, in resize_image  *
        new_image = tf.image.resize_images(

    ValueError: 'images' must have either 3 or 4 dimensions.


In [16]:
num_detections = int(detections.pop('num_detections'))
detections

{'detection_boxes': <tf.Tensor: shape=(1, 100, 4), dtype=float32, numpy=
 array([[[0.6818859 , 0.22158414, 0.8492995 , 0.4246689 ],
         [0.05995774, 0.5043771 , 0.70022815, 0.63283694],
         [0.65753734, 0.555839  , 0.8007293 , 0.77737266],
         [0.06248081, 0.29679847, 0.66723496, 0.44353694],
         [0.6091651 , 0.22092384, 0.8262366 , 0.4117896 ],
         [0.49674165, 0.22255148, 0.84230995, 0.4403028 ],
         [0.60490865, 0.5576683 , 0.7686407 , 0.7841114 ],
         [0.7384535 , 0.2023287 , 0.8804687 , 0.45657587],
         [0.        , 0.        , 1.        , 1.        ],
         [0.65699077, 0.21087313, 0.8639606 , 0.36674196],
         [0.08375317, 0.56503403, 0.7018735 , 0.67548954],
         [0.14521982, 0.8885613 , 0.54494846, 1.        ],
         [0.02948353, 0.3373245 , 0.6803429 , 0.47124803],
         [0.68687975, 0.5119877 , 0.8426802 , 0.7569171 ],
         [0.90026   , 0.57883793, 1.        , 0.86532575],
         [0.        , 0.        , 1.      

In [10]:
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
            for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.3,
            agnostic_mode=False)

cv.imshow('object detection',  cv.resize(image_np_with_detections, (800, 600)))
cv.waitKey(0)

-1